### Basic RAG retriever using Langchain and Anthropic model

../notebooks/data/cleaned20242108.csv

In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
os.environ["LANGCHAIN_PROJECT"] = "Trials"

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
db = SQLDatabase.from_uri("sqlite:////Users/main/Desktop/database/llm.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM restaurants LIMIT 10;")

sqlite
['restaurants', 'restaurants_madrid']


"[(0, 'OPERATIONAL', 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', '#909CE1', 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'VP Jardín de Recoletos', 'ChIJ0U9LFZooQg0RgdlcHxZdkek', 4.5, 'ChIJ0U9LFZooQg0RgdlcHxZdkek', 'GOOGLE', 611.0, 'Calle de Gil de Santivañes, 6, Madrid', 40.4223075, -3.689168100000001, 40.42368518029149, -3.687793119708498, 40.42098721970849, -3.690491080291502, 1, 'C8C6+W8 Madrid, Spain', '8CGRC8C6+W8', None, None), (1, 'OPERATIONAL', 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', '#FF9E67', 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'New York Burger', 'ChIJ2fFEjZooQg0RbySrRH-t_nk', 4.5, 'ChIJ2fFEjZooQg0RbySrRH-t_nk', 'GOOGLE', 7357.0, 'Calle de Recoletos, 4, Madrid', 40.422131, -3.6908463, 40.4234758302915, -3.689437969708498, 40.4207778697085, -3.692135930291502, 1, 'C8C5+VM Madrid, Spain', '8CGRC8C5+VM', 2.0, None), (2, 'CLOSED_TEMPORARILY', 'h

In [3]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db="sqlite:////Users/main/Desktop/database/llm.db")
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "how many restaurants are in the restaurants_madrid table? dont use double notation mark in the query for getting total_restaurants, toherwise you will not give me a result"})

ValidationError: 1 validation error for QuerySQLDataBaseTool
db
  instance of SQLDatabase expected (type=type_error.arbitrary_type; expected_arbitrary_type=SQLDatabase)

In [5]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many restaurants are in the restaurants_madrid table? if you find a grammar mistake fixed it and return me the result of the query"})

"I apologize for the confusion in the previous response. It seems there was an error in executing the SQL query. The correct SQL query to answer your question would be:\n\n```sql\nSELECT COUNT(*) AS restaurant_count FROM restaurants_madrid\n```\n\nHowever, since we don't have the actual result of this query, I cannot provide you with the exact number of restaurants in the restaurants_madrid table. \n\nTo properly answer your question, we would need to run this query against the database containing the restaurants_madrid table and retrieve the result. Once we have the result, we could tell you exactly how many restaurants are in the table.\n\nIf you have access to the database, you can run this query to get the count of restaurants. If you don't have direct access, you may need to ask someone with database access to run this query for you."